In [1]:
import os
import sys
sys.path.append("../")

import pandas as pd
import numpy as np
import datetime as dt

# from src.etl import get_meta_query, read_meta, write_to_snowflake
from src.utils.parser import parse_args, parse_config
from src.utils.vault import get_vault_secret

from pyathena import connect

In [2]:
# if '__file__' in globals():
#     cwd = os.path.dirname(__file__)
# else:
#     cwd = os.getcwd()
config = parse_config(os.path.abspath(os.path.join(os.getcwd(), "../src/settings.yml")))
# args = parse_args()

athena_args = get_vault_secret(
    **config["vault"],
    path=os.path.join(
        config["vault"]["stack"], config["environment"], config["vault"]["path_end"]
    ),
)

In [3]:
athena_ctx = connect(
    aws_access_key_id=athena_args["gateway_aws_iam_access_id"],
    aws_secret_access_key=athena_args["gateway_aws_iam_access_key"],
    s3_staging_dir=athena_args[" S3 Output Location"],
    region_name=athena_args[" AWS Region"],
)

In [11]:
n_extremes = 5

last_processed = "2000-1-1 15:35:04.518 UTC"
last_processed = pd.read_parquet("../data/raw", columns=["processed_on"]).max()[0]
last_processed = dt.datetime.strptime(last_processed, "%Y-%m-%d %H:%M:%S.%f %Z").date()
from_date = last_processed - dt.timedelta(days=last_processed.weekday())

channel_like = "register://electricity/0/activepower/sumli?avg=15"
# channel_like = "register://electricity/0/activepower/l_?avg=15"

query = f"""
SELECT *
FROM
(
    SELECT
        t1.boxid,
        t1.channelid,
        t1.value,
        t1.year,
        t1.week,
        ROW_NUMBER() OVER (PARTITION BY (t1.boxid, t1.channelid, t1.year, t1.week) ORDER BY (t1.value) DESC) AS top,
        -ROW_NUMBER() OVER (PARTITION BY (t1.boxid, t1.channelid, t1.year, t1.week) ORDER BY (t1.value) ASC) AS bottom,
        CURRENT_TIMESTAMP AS processed_on 
     FROM (
        SELECT 
            t0.boxid AS boxid,
            t0.channelid AS channelid,
            t0.value AS value,
            YEAR_OF_WEEK(t0.timestamp) AS year,
            WEEK(t0.timestamp) AS week
        FROM "enxp401-src-dali".enxp401_src_dali_readings t0
        WHERE t0.channelid like '{channel_like}'
         AND t0.timestamp >= DATE('{from_date}')
         AND t0.timestamp < DATE_ADD('DAY', -DAY_OF_WEEK(CURRENT_DATE), CURRENT_DATE)
--         AND t0.boxid IN ('075.547-1', '069.509-1')
--         LIMIT 100
         ) t1
) t2
WHERE t2.top <= {n_extremes} OR t2.bottom >=-{n_extremes}
"""

In [12]:
%%time
df_query = pd.read_sql(sql=query, con=athena_ctx)

CPU times: user 435 ms, sys: 2.2 s, total: 2.63 s
Wall time: 6.81 s


In [13]:
df_query #.query("year == 2020").sort_values(["year", "week"])

,boxid,channelid,value,year,week,top,bottom,processed_on


In [10]:
# df_query.to_parquet("../data/raw/", partition_cols=["year", "week"])

In [9]:
pd.read_parquet("../data/raw")

,boxid,channelid,value,top,bottom,processed_on,year,week
0,ZL.TEST_FW-2,register://electricity/0/activepower/sumli?avg=15,0.000000e+00,1,-1,2021-06-07 18:24:06.112 UTC,2019,15
1,ZL.TEST_FW-2,register://electricity/0/activepower/sumli?avg=15,0.000000e+00,2,-2,2021-06-07 18:24:06.112 UTC,2019,15
2,ZL.TEST_FW-2,register://electricity/0/activepower/sumli?avg=15,0.000000e+00,3,-3,2021-06-07 18:24:06.112 UTC,2019,15
3,ZL.TEST_FW-2,register://electricity/0/activepower/sumli?avg=15,0.000000e+00,4,-4,2021-06-07 18:24:06.112 UTC,2019,15
4,ZL.TEST_FW-2,register://electricity/0/activepower/sumli?avg=15,0.000000e+00,5,-5,2021-06-07 18:24:06.112 UTC,2019,15
...,...,...,...,...,...,...,...,...
3921589,ZLE.711724-1,register://electricity/0/activepower/sumli?avg=15,-6.963788e-08,666,-5,2021-06-07 18:24:06.112 UTC,2021,9
3921590,ZLE.711724-1,register://electricity/0/activepower/sumli?avg=15,-6.963788e-08,667,-2,2021-06-07 18:24:06.112 UTC,2021,9
3921591,ZLE.711724-1,register://electricity/0/activepower/sumli?avg=15,-6.963788e-08,668,-3,2021-06-07 18:24:06.112 UTC,2021,9
3921592,ZLE.711724-1,register://electricity/0/activepower/sumli?avg=15,-6.963788e-08,669,-4,2021-06-07 18:24:06.112 UTC,2021,9
